In [1]:
%reset
#%qtconsole
%load_ext autoreload
%autoreload 2
%matplotlib inline 
import matplotlib.pyplot as plt

import numpy as np
import matplotlib as mpl
from scipy import stats


from scipy.interpolate import interp1d
import pandas as pd


import pickle

from sklearn.preprocessing import StandardScaler  
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPRegressor
import sklearn
from sklearn.model_selection import GridSearchCV
import itertools
import warnings


Once deleted, variables cannot be recovered. Proceed (y/[n])? y


# Read all simulation results, train parameter scaler

In [2]:
folder='./../../../DataAndScripts/unstructured_scripts/simulation_results/'
results=np.loadtxt(folder+'results_0.txt');
results=np.concatenate((results,np.loadtxt(folder+'results_1.txt')));
results=np.concatenate((results,np.loadtxt(folder+'results_2.txt')));
results=np.concatenate((results,np.loadtxt(folder+'results_3.txt')));
results=np.concatenate((results,np.loadtxt(folder+'results_4.txt')));
sim_param_all=results[:,0:15]
moments_of_r_sim_all=results[:,15:20]
sim_convergence_all=results[:,20:24]
sim_decay_all=results[:,26]/results[:,27]

sim_g_E=(sim_param_all[:,2])
sim_g_I=(sim_param_all[:,3])
sim_beta=(sim_param_all[:,4])
sim_CV_K=(sim_param_all[:,7])
sim_sigma_Lambda_over_Lambda=(sim_param_all[:,10])
sim_J=(sim_param_all[:,11])
sim_r_X=(sim_param_all[:,12])
sim_ell=(sim_param_all[:,13])


scaler = StandardScaler()  

X=np.vstack((sim_g_E,sim_g_I,sim_beta,sim_CV_K,sim_sigma_Lambda_over_Lambda,
             sim_J,sim_r_X,sim_ell)).T
X[:,2::]=np.log10(X[:,2::])

Y=np.zeros(np.shape(moments_of_r_sim_all))
Y[:,:]=moments_of_r_sim_all[:,:]
Y[:,4]=np.sign(Y[:,4])*np.sqrt(np.abs(Y[:,4]))
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, train_size=0.80, test_size=0.20, random_state=123)

# center training dataset
scaler.fit(X_train) 

with open('model_files/scaler.pkl','wb') as f:
    pickle.dump(scaler,f)


# Design fits

## Write simulation parameters with set J and CVopto

In [3]:
Possible_idx_species=[0,1]
Possible_log10_CVopto=np.linspace(-1,1,20)
Possible_log10_J=np.linspace(-5,-2.5,20)
nRep=101

            
count=0;
for idx_species in [0,1]:
    for idx_S in range(len(Possible_log10_CVopto)):
        log10_CVopto=Possible_log10_CVopto[idx_S]
        for idx_J in range(len(Possible_log10_J)):
            log10_J=Possible_log10_J[idx_J]
            data=np.ones((1,4));
            data[:,0]=idx_species
            data[:,1]=log10_CVopto
            data[:,2]=log10_J
            data[:,3]=nRep

            if count==0:
                DATA=data;
            if count>0:
                DATA=np.vstack((DATA,data));
            count=count+1

len(DATA)

df = pd.DataFrame(DATA,columns=['idx_species','log10_CVopto','log10_J','nRep'])
file_name='simulation_files/simulation_param_Fixed_J_and_CVopto.txt'
df.to_csv(file_name, sep='\t')

## Write simulation parameters with set J and CVopto

In [4]:
Possible_idx_species=[0,1]
possible_g_E=np.linspace(3,10,20)

            
count=0;
for idx_species in [0,1]:
    for idx_g_E in range(len(possible_g_E)):
        g_E=possible_g_E[idx_g_E];
        possible_g_I=np.arange(2,g_E,np.diff(possible_g_E)[0])
        for idx_g_I in range(len(possible_g_I)):
            g_I=possible_g_I[idx_g_I];

            data=np.ones((1,4));
            data[:,0]=idx_species
            data[:,1]=g_E
            data[:,2]=g_I
            data[:,3]=nRep

            if count==0:
                DATA=data;
            if count>0:
                DATA=np.vstack((DATA,data));
            count=count+1

print(len(DATA))

df = pd.DataFrame(DATA,columns=['idx_species','g_E','g_I','nRep'])
file_name='simulation_files/simulation_param_Fixed_gs.txt'
df.to_csv(file_name, sep='\t')

500


# Train perceptron to predict optogenetic response statistics

In [ ]:
sim_param_all=results[:,0:15]
moments_of_r_sim_all=results[:,15:20]
sim_convergence_all=results[:,20:24]
sim_decay_all=results[:,26]/results[:,27]

sim_g_E=(sim_param_all[:,2])
sim_g_I=(sim_param_all[:,3])
sim_beta=(sim_param_all[:,4])
sim_CV_K=(sim_param_all[:,7])
sim_sigma_Lambda_over_Lambda=(sim_param_all[:,10])
sim_J=(sim_param_all[:,11])
sim_r_X=(sim_param_all[:,12])
sim_ell=(sim_param_all[:,13])


scaler = StandardScaler()  

X=np.vstack((sim_g_E,sim_g_I,sim_beta,sim_CV_K,sim_sigma_Lambda_over_Lambda,
             sim_J,sim_r_X,sim_ell)).T
X[:,2::]=np.log10(X[:,2::])

Y=np.zeros(np.shape(moments_of_r_sim_all))
Y[:,:]=moments_of_r_sim_all[:,:]
Y[:,4]=np.sign(Y[:,4])*np.sqrt(np.abs(Y[:,4]))
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, train_size=0.80, test_size=0.20, random_state=123)

# center training dataset
scaler.fit(X_train)  
X_train = scaler.transform(X_train)  
# apply same transformation to test data
X_test = scaler.transform(X_test)

# Use optimization in the plot reults file to find best network structure
mlp_regressor  = MLPRegressor(random_state=123,
                          activation='relu',
                          hidden_layer_sizes=(100, 150, 50),verbose=True)
mlp_regressor.fit(X_train, Y_train)

Y_preds = mlp_regressor.predict(X_test)

#print(Y_preds[:5])
#print(Y_test[:5])

print('Test R^2 Score : %.3f'%mlp_regressor.score(X_test, Y_test)) ## Score method also evaluates accuracy for classification models.
print('Training R^2 Score : %.3f'%mlp_regressor.score(X_train, Y_train))

Iteration 1, loss = 299.53041492
Iteration 2, loss = 107.43174408


In [ ]:
with open('model_files/model.pkl','wb') as f:
    pickle.dump(mlp_regressor,f)

# Train perceptron to predict chaos

In [ ]:
sim_param_all=results[:,0:15]
moments_of_r_sim_all=results[:,15:20]
sim_convergence_all=results[:,20:24]
sim_decay_all=results[:,26]/results[:,27]

sim_g_E=(sim_param_all[:,2])
sim_g_I=(sim_param_all[:,3])
sim_beta=(sim_param_all[:,4])
sim_CV_K=(sim_param_all[:,7])
sim_sigma_Lambda_over_Lambda=(sim_param_all[:,10])
sim_J=(sim_param_all[:,11])
sim_r_X=(sim_param_all[:,12])
sim_ell=(sim_param_all[:,13])


scaler = StandardScaler()  

X=np.vstack((sim_g_E,sim_g_I,sim_beta,sim_CV_K,sim_sigma_Lambda_over_Lambda,
             sim_J,sim_r_X,sim_ell)).T
X[:,2::]=np.log10(X[:,2::])

Y=np.zeros(np.shape(moments_of_r_sim_all))
Y[:,:]=moments_of_r_sim_all[:,:]
Y[:,4]=np.sign(Y[:,4])*np.sqrt(np.abs(Y[:,4]))
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, train_size=0.80, test_size=0.20, random_state=123)

# center training dataset
scaler.fit(X_train)  
X_train = scaler.transform(X_train)  
# apply same transformation to test data
X_test = scaler.transform(X_test)

# Use optimization in the plot reults file to find best network structure
mlp_regressor  = MLPRegressor(random_state=123,
                          activation='relu',
                          hidden_layer_sizes=(100, 150, 50),verbose=True)
mlp_regressor.fit(X_train, Y_train)

Y_preds = mlp_regressor.predict(X_test)

#print(Y_preds[:5])
#print(Y_test[:5])

print('Test R^2 Score : %.3f'%mlp_regressor.score(X_test, Y_test)) ## Score method also evaluates accuracy for classification models.
print('Training R^2 Score : %.3f'%mlp_regressor.score(X_train, Y_train))

In [ ]:
with open('model_files/chaos_model.pkl','wb') as f:
    pickle.dump(mlp_regressor,f)

# Next step: Run sim_Fixed_J_and_CVopto.py and sim_Fixed_gs.py